In [1]:
import vault_client
import getpass

from os.path import expanduser
import json

def get_config():
    home = expanduser("~")
    
    with open(home + '/config.json') as f:
        config_data = f.read()
    config = json.loads(config_data)
    return config

yav = vault_client.instances.Production(
    rsa_login='miptgirl', 
    authorization='OAuth {}'.format(get_config()['vault_api_token']),)

ver = 'ver-01dssnc1zdezgbfd48xhj9j9xn'
YT_TOKEN = yav.get_version(ver)['value']['yt_token']
YQL_TOKEN = yav.get_version(ver)['value']['yql_token']
SOLOMON_TOKEN = yav.get_version(ver)['value']['solomon_token']

def df_to_wiki(df, show_index = False):
    print('#|\n ||' + df.to_csv(index = show_index, sep = '|').replace('|', ' | ').replace('\n', '|| \n ||')[:-2] + '|#')

In [2]:
import io
import time
import pandas as pd
import datetime

HOST = 'http://mtgray03k.yandex.ru:8123'
HOST = 'http://mtlog01-01-1.yandex.ru:8123'
HOST = 'http://clickhouse.metrika.yandex.net:8123'
#HOST = 'http://mtsmart001-1.yandex.ru:8123'
HOST = 'http://mtch01k.metrika.yandex.net:8123'

import requests
import http.client  # or http.client if you're on Python 3
http.client._MAXHEADERS = 100000

def get_clickhouse_data(query, host = HOST, connection_timeout = 1500):
    NUMBER_OF_TRIES = 30
    DELAY = 10
    
    for i in range(NUMBER_OF_TRIES):
        r = requests.post(host, 
                          params = {'timeout_before_checking_execution_speed': 120}, 
                          timeout = connection_timeout, data = query,
                          auth = ('miptgirl', '1QwBgINk')
                         )
        if (r.status_code == 200) and ('DB:Exception' not in r.text):
            return r.text
        else:
            print('ATTENTION: try #%d failed' % i)
            if i != (NUMBER_OF_TRIES-1):
                #print query
                print(r.text)
                time.sleep(DELAY*(i+1))
            else:
                raise ValueError(r.text)
        
def get_clickhouse_df(query, host = HOST, connection_timeout = 1500):
    data = get_clickhouse_data(query, host, connection_timeout) 
    df = pd.read_csv(io.StringIO(data), sep = '\t')
    return df

def upload(table, content, host=HOST):
    '''Uploads data to table in ClickHous'''
    content = content.encode('utf-8')
    query_dict = {
             'query': 'INSERT INTO ' + table + ' FORMAT TabSeparatedWithNames '
        }
    r = requests.post(host, data=content, params=query_dict, auth = ('miptgirl', '1QwBgINk'))
    result = r.text
    if r.status_code == 200:
        return result
    else:
        raise ValueError(r.text)

In [52]:
q = """
select extract(x, '\\(([0-9]+)\\)')
from (select arrayJoin(['(0)', '(10)']) as x)
format TSVWithNames
""".format(date_start = '2020-09-10', date_end = '2020-09-10')

q = """
select match(x, '^https?://yandex.[\\\w\\\.]{{2,}}/?$') as ya
from (select arrayJoin(['http://yandex.ru', 'http://bla.com']) as x)
format TSVWithNames
""".format(date_start = '2020-09-10', date_end = '2020-09-10')



In [64]:
q = """
select match(x, '^([a-z0-9]([a-z0-9_\\\-]{{0,61}}[a-z0-9_])?\\\.)+(xn\\\-\\\-[a-z0-9\\\-]{{2,}}|[a-z]{{2,}})$') as ya
from (select arrayJoin(['yandex.ru', 'bla.com']) as x)
format TSVWithNames
""".format(date_start = '2020-09-10', date_end = '2020-09-10')


q = r"""
select match(x, '(\\w+)') as ya
from (select arrayJoin(['yandex', 'bla']) as x)
format TSVWithNames
""".format(date_start = '2020-09-10', date_end = '2020-09-10')

In [65]:
get_clickhouse_df(q, 'http://mtgiga001-1.metrika.yandex.net:8123')

,ya
0,1
1,1


In [66]:
get_clickhouse_df(q, 'http://mtgiga001-1t.metrika.yandex.net:8123')

,ya
0,1
1,1


In [39]:
print('\\\\w')

\\w
